# Prática WTSS
<hr style="border:2px solid #0077b9;">

- Rolf Simoes
- Gilberto Ribeiro
- Vitor Gomes

## Agenda
<hr style="border:2px solid #0077b9;">
- Arquitetura de acesso a dados

- Comandos básicos do WTSS

- Python integration

# Arquitetura de acesso a dados
<hr style="border:2px solid #0077b9;">

<img src="./imgs/arquitetura.png" alt="Arquitetura 1" style="width: 1200px;"/>


## Comandos básicos do WTSS

- ```wtss.list_coverages()```: Obtém uma lista das coverages disponíveis.
- ```wtss.describe_coverage()```: Obtém o meta-dados de uma coverage.
- ```wtss.time_series()```: Obtém uma série temporal para uma localização e intervalo de tempo.

O cliente WTSS Python está disponível em https://github.com/e-sensing/wtss.py

## WTSS: Listando coverages
<hr style="border:1px solid #0077b9;">

```wtss.list_coverages()```

In [ ]:
# WTSS python client: Access to data & metadata
from wtss import wtss

# connect to e-Sensing server
w = wtss("http://www.dpi.inpe.br/tws")

# print the available data sets
cv_list = w.list_coverages()
for cv_name in cv_list["coverages"]:
    print(cv_name)

## WTSS: Descrevendo coverages
<hr style="border:1px solid #0077b9;">

```wtss.describe_coverage(cv_name)```

In [ ]:
# explore a WTSS data set
cv_scheme = w.describe_coverage("mod13q1_512")

# format output
print("SciDB array: {}. {}({})\n".format(cv_scheme["name"],
                                         cv_scheme['description'], 
                                         cv_scheme['detail']))
print("Linha do tempo:\n{}...{}\n".format(cv_scheme['timeline'][0:3], 
                                          cv_scheme['timeline'][-3:]))
print("Atributos:")
for el in cv_scheme['attributes']:
    att = cv_scheme['attributes'][el]
    print("{}: {}. Tipo: {}. Fator de escala: {}".format(el,
                                                         att['description'],
                                                         att['datatype'], 
                                                         att['scale_factor']))

## WTSS: Obtendo séries temporais
<hr style="border:1px solid #0077b9;">

```wtss.time_series(coverage, attributes, latitude, longitude, start_date=None, end_date=None)```

In [ ]:
import pandas as pd
# we are interested in observing land dynamics at
latitude  = -14.919100049
longitude = -59.11781088
# get time series of a point
ts = w.time_series("mod13q1_512", ("ndvi", "evi"), latitude, longitude)
# build a data frame made of vegetation indexes
ndvi = pd.Series(ts["ndvi"], index = ts.timeline) * \
        cv_scheme['attributes']['ndvi']['scale_factor']
evi  = pd.Series(ts["evi"],  index = ts.timeline) * \
        cv_scheme['attributes']['evi']['scale_factor']
vidf = pd.DataFrame({'ndvi': ndvi, 'evi': evi})
vidf[0:5]

In [ ]:
# Let's show the choosen location on a map
from tsmap import *
location = {'lon': longitude, 'lat': latitude}
createTSMap(location, vidf, 4)

## Visualização de dados
<hr style="border:1px solid #0077b9;">

O Python fornece ferramentas para visualização de dados científicos.

In [ ]:
%matplotlib inline
import matplotlib
from cycler import cycler
matplotlib.style.use('ggplot')
# Updating the default matplotlib colors
colors = cycler(u'color', [u'#74c476',u'#6baed6',u'#d62728', \
                           u'#ff7f0e', u'#756bb1'])
matplotlib.rcParams['axes.prop_cycle'] = colors
# Time series visualization
fig, ax = matplotlib.pyplot.subplots(figsize = (15, 5))
ax.plot()
vidf['ndvi'].plot()
vidf['evi'].plot()
ax.legend()
fig.autofmt_xdate()

## Análise dos dados
<hr style="border:1px solid #0077b9;">

### Ajuste de modelo linear
<hr>

Um jeito simples de revelar uma tendência geral é ajustar uma reta nos dados.

In [ ]:
from linearmodel import *
# fit a line to the vegetation indexes
vidf['ndvi_lm'] = fitline(vidf['ndvi'])
vidf['evi_lm'] = fitline(vidf['evi'])
# plot
fig, ax = matplotlib.pyplot.subplots(figsize = (15, 5))
ax.plot()
vidf['ndvi'].plot()
vidf['evi'].plot()
vidf['ndvi_lm'].plot()
vidf['evi_lm'].plot()
ax.legend()
fig.autofmt_xdate()
#vidf[0:5]

### Decomposição por Fourier
<hr>
É possível decompor uma série temporal em um conjunto de senóides por Fourier. Essas funções nos dão a amplitude, a frequência e a fase de cada harmônica do sinal. Geralmente, altas frequências estão associadas a ruído. Removendo as funções de alta frequência ajudam a obter um sinal mais suave.

In [ ]:
from fourier import *
# filter the vi
vidf['ndvi_ff'] = fourierfilter(vidf['ndvi'], 200)
vidf['evi_ff'] = fourierfilter(vidf['evi'], 50)
# plot
fig, ax = matplotlib.pyplot.subplots(figsize = (15, 5))
ax.plot()
vidf['ndvi'].plot()
vidf['evi'].plot()
vidf['ndvi_ff'].plot()
vidf['evi_ff'].plot()
ax.legend()
fig.autofmt_xdate()
#vidf[0:5]

### Filtro Whittaker
<hr>

O filtro Whitakker é uma combinação linear de pontos contíguos na série temporal.

In [ ]:
from whittaker import *
# filter the vi
vidf['ndvi_wf'] = pd.Series(whittaker_filter(ndvi,1000), index = ts.timeline)
vidf['evi_wf']  = pd.Series(whittaker_filter(evi,1), index = ts.timeline)
# plot
fig, ax = matplotlib.pyplot.subplots(figsize = (15, 5))
ax.plot()
vidf['ndvi'].plot()
vidf['evi'].plot()
vidf['ndvi_wf'].plot()
vidf['evi_wf'].plot()
ax.legend()
fig.autofmt_xdate()
#vidf[0:5]

## Classificação
<hr style="border:1px solid #0077b9;">

### Dynamic Time Warping
<hr>

Dynamic Time Warping (DTW) é um algoritmo de casamento de padrões. É uma função de distância que distorce o tempo para melhor ajustar duas séries temporais.

#### Padrões

Nossos padrões são séries temporais idealizadas para um ano de [floresta](floresta) e [cerrado](https://en.wikipedia.org/wiki/Cerrado). Esses padrões foram obtidos utilizando Modelo Aditivo Generalizado [GAM, da sigla em inglês](https://en.wikipedia.org/wiki/Generalized_additive_model) de um grande número de séries temporais amostradas por especialistas.

In [ ]:
from dtw import *
from tools import *

# open the pattern file
patterns_ts = pd.read_json("examples/patterns.json", orient='records')
# update timeline type from str to datetime
patterns_ts["timeline"] = pd.to_datetime(patterns_ts["timeline"])
plot_time_series(patterns_ts)

#### Amostras

Nossas amostras usadas para classificação consiste de um arquivo CSV com 10 pontos.

In [ ]:
# read the sample file
samples = pd.read_csv("examples/samples.csv")
samples

Agora obtemos as séries temporais usando o serviço WTSS para cada uma das amostras.

In [ ]:
# wtss_get_time_series is implemented in 'tools.py'
samples_ts = wtss_get_time_series(samples)

# rescale vegetation indexes
samples_ts["ndvi"] *= cv_scheme['attributes']['ndvi']['scale_factor']
samples_ts["evi"] *= cv_scheme['attributes']['evi']['scale_factor']

samples_ts[0:5]
plot_time_series(samples_ts)

#### Classificação

É hora de classificar as 10 amostras usando os padrões e o método DTW. Nós fazemos isso calculando as distâncias DTW entre cada padrão e as amostras. Nós então atribuímos o rótulo do padrão que tiver menor distância DTW para cada uma das amostras.

In [ ]:
# classify using DTW
classification = classifier_1nn(patterns_ts, samples_ts)

# print the classification results
classification

#### Resultados da classificação

Os resultados anteriores mostram que o DTW + classificador vizinho mais próximo fizeram um bom trabalho. Nós classificamos corretamente 9 das 10 amostras de séries temporais. A amostra número 3 foi classificada incorretamente. Vamos olhar mais de perto o que aconteceu.

In [ ]:
# let's see what happens with sample #3
plot_time_series(pd.concat([samples_ts[samples_ts["id"].isin([3])], patterns_ts]))

# Considerações finais
<hr style="border:2px solid #0077b9">

Uso do WTSS, visualização, filtragem e classificação de séries temporais.

Os exemplos utilizados fizeram uso dos seguintes módulos do Python (disponíveis no notebook do curso):
- Map
  - [tsmap.py](./tsmap.py) Exibir mapas em notebooks do Jupyter.
- Filtro
  - [linearmodel.py](./linearmodel.py) Ajuste linear em séries temporais.
  - [fourier.py](./fourier.py) Filtro de séries temporais usando Transformada Rápida de Fourier.
  - [whittaker.py](./whittaker.py) Filtro de séries temporais usando o amaciador Whittaker.
- Classificação: 
  - [dtw.py](./dtw.py) Implementação do DTW.
  - [tools.py](./tools.py) Funções utilitárias para obter e plotar séries temporais.

# Obrigado!


Baseado no notebook: https://github.com/e-sensing/wgiss-py-webinar
Alber Sánchez
alber.ipia @ inpe.br
